# Introduction to Genetic Algorithms (GAs)

In this lesson, we will build a simple genetic algorithm (GA) from scratch. We will use a guided example where our goal is to **maximize** the following fitness function:


$$F(x, y, z) = (x+1) \times (y+1) \times (z+1)$$

Here, $x, y, z$ are integer variables, each ranging from 0 to 3.

Below is our initial population of four individuals. 

| x | y | z |
|---|---|---|
| 1 | 0 | 3 |
| 1 | 2 | 0 |
| 1 | 3 | 1 |
| 1 | 0 | 0 |

## Ouline of the basic genetic algorithm
- **Start**: Generate a random population of *n* chromosomes (suitable solutions for the problem).
- **Fitness**: Evaluate the fitness *f(x)* of each chromosome *x* in the population.
- **New population**: Create a new population by repeating the following steps until the new population is complete:
  - **Selection**: Select two parent chromosomes from the population according to their fitness (the better the fitness, the higher the chance of being selected).
  - **Crossover**: With a crossover probability, cross over the parents to form a new offspring (children). If no crossover was performed, the offspring is an exact copy of the parents.
  - **Mutation**: With a mutation probability, mutate the new offspring at each locus (position in the chromosome).
  - **Accepting**: Place the new offspring in the new population.
- **Replace**: Use the newly generated population for a further run of the algorithm.
- **Test**: If the end condition is satisfied, stop and return the best solution in the current population.
- **Loop**: Go to step 2.




Let's dive in!

In [1]:
# Importing required libraries
import random

# Set a random seed for reproducibility (optional)
random.seed(42)

# Mapping for binary encoding (each number 0-3 uses 2 bits)
bin_map = {0: '00', 1: '01', 2: '10', 3: '11'}

def encode_individual(individual):
    """
    Convert an individual (tuple of (x, y, z)) to its binary string representation.
    Each variable is represented by 2 bits.
    """
    return "".join(bin_map[val] for val in individual)

def decode_individual(binary_str):
    """
    Convert a 6-bit binary string back to a tuple (x, y, z) of decimal values.
    """
    x = int(binary_str[0:2], 2)
    y = int(binary_str[2:4], 2)
    z = int(binary_str[4:6], 2)
    return (x, y, z)

def fitness(individual):
    """
    Evaluate the fitness of an individual using:
    
    F(x, y, z) = (x+1) * (y+1) * (z+1)
    """
    x, y, z = individual
    return (x + 1) * (y + 1) * (z + 1)

def two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8):
    """
    Perform two-point crossover on two binary strings.
    cp1 and cp2 define the crossover cut points.
    Returns two offspring binary strings.
    """
    if random.random() > crossover_rate:
        # No crossover occurs; offspring are copies of the parents.
        return parent1, parent2
    
    offspring1 = parent1[:cp1] + parent2[cp1:cp2] + parent1[cp2:]
    offspring2 = parent2[:cp1] + parent1[cp1:cp2] + parent2[cp2:]
    return offspring1, offspring2

def mutate(binary_str, mutation_index=None):
    """
    Flip a random bit in the binary string (or at a specified index).
    """
    bits = list(binary_str)
    if mutation_index is None:
        mutation_index = random.randint(0, len(bits) - 1)
    # Flip the bit
    bits[mutation_index] = '1' if bits[mutation_index] == '0' else '0'
    return "".join(bits)


### Step 1: Generate a random population of n chromosomes (suitable solutions for the problem).
In this case we are not generating a random population since we are given it by the exercise.

We still need to go from the decimal expression of each triple of integers to the binary encoding. 

In [44]:
population_decimal = [
    (1, 0, 3),
    (1, 2, 0),
    (1, 3, 1),
    (1, 0, 0)
]

In [45]:
population_binary = [ encode_individual(x) for x in population_decimal]
population_binary

['010011', '011000', '011101', '010000']

### Step2: Evaluate the fitness f(x) of each chromosome x in the population.

In [46]:
print("Initial Population (Decimal, Binary, Fitness):")
for ind in population_decimal:
    encoded = encode_individual(ind)
    print(f"{ind} -> {encoded} -> Fitness: {fitness(ind)}")

Initial Population (Decimal, Binary, Fitness):
(1, 0, 3) -> 010011 -> Fitness: 8
(1, 2, 0) -> 011000 -> Fitness: 12
(1, 3, 1) -> 011101 -> Fitness: 14
(1, 0, 0) -> 010000 -> Fitness: 8


### Step 3: Create a new population by repeating the following steps until the new population is complete:
- **Selection**: Select two parent chromosomes from the population according to their fitness (the better the fitness, the higher the chance of being selected).
- **Crossover**: With a crossover probability, cross over the parents to form a new offspring (children). If no crossover was performed, the offspring is an exact copy of the parents.
- **Mutation**: With a mutation probability, mutate the new offspring at each locus (position in the chromosome).
- **Accepting**: Place the new offspring in the new population.

In our case we are creating two pairs of parents, without loooking at their fitness, just because that is what the exercise says. So we are not selecting according to the fitness.

In [47]:
# For Generation 1, we choose fixed pairings:
# Pair 1: Individual 1 and Individual 3 (indices 0 and 2)
# Pair 2: Individual 2 and Individual 4 (indices 1 and 3)

# Pair 1:
parent1 = population_binary[0]  # e.g., '010011'
parent2 = population_binary[2]  # e.g., '011101'
offspring1, offspring2 = two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8)

# Apply mutation on both offspring
offspring1 = mutate(offspring1)
offspring2 = mutate(offspring2)

# Pair 2:
parent3 = population_binary[1]  # e.g., '011000'
parent4 = population_binary[3]  # e.g., '010000'
offspring3, offspring4 = two_point_crossover(parent3, parent4, cp1=2, cp2=4, crossover_rate=0.8)
offspring3 = mutate(offspring3)
offspring4 = mutate(offspring4)

# New Generation 1 population in binary and decoded in decimal
P1_binary = [offspring1, offspring2, offspring3, offspring4]
P1_decimal = [decode_individual(ind) for ind in P1_binary]

print("Generation 1 (P₁):")
for i, ind in enumerate(P1_decimal):
    print(f"Offspring {i+1}: {ind} -> Binary: {P1_binary[i]} -> Fitness: {fitness(ind)}")


Generation 1 (P₁):
Offspring 1: (3, 3, 3) -> Binary: 111111 -> Fitness: 30
Offspring 2: (3, 0, 1) -> Binary: 110001 -> Fitness: 48
Offspring 3: (1, 0, 2) -> Binary: 010010 -> Fitness: 8
Offspring 4: (1, 2, 2) -> Binary: 011010 -> Fitness: 8


## Discussion of Generation 1

In Generation 1, after applying two-point crossover and mutation, we obtained new offspring. Each offspring was decoded back to the decimal representation and evaluated using our fitness function:

$$F(x, y, z) = (x+1) \times (y+1) \times (z+1)$$

Observe the fitness values and the changes in the chromosomes. This demonstrates how genetic operators introduce variability into the population.

Next, we will perform another GA iteration to generate Generation 2.


In [30]:
# For Generation 2, we will use the Generation 1 population (P1_binary)
# New fixed pairings for diversity:
# Pair 1: Offspring 1 and Offspring 4
# Pair 2: Offspring 2 and Offspring 3

# Pair 1:
parent1 = P1_binary[0]
parent2 = P1_binary[3]
offspring1, offspring2 = two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8)
offspring1 = mutate(offspring1)
offspring2 = mutate(offspring2)

# Pair 2:
parent3 = P1_binary[1]
parent4 = P1_binary[2]
offspring3, offspring4 = two_point_crossover(parent3, parent4, cp1=2, cp2=4, crossover_rate=0.8)
offspring3 = mutate(offspring3)
offspring4 = mutate(offspring4)

# New Generation 2 population in binary and decoded in decimal
P2_binary = [offspring1, offspring2, offspring3, offspring4]
P2_decimal = [decode_individual(ind) for ind in P2_binary]

print("Generation 2 (P₂):")
for i, ind in enumerate(P2_decimal):
    print(f"Offspring {i+1}: {ind} -> Binary: {P2_binary[i]} -> Fitness: {fitness(ind)}")

# Identify and display the best solution in Generation 2
best_solution = max(P2_decimal, key=fitness)
print("\nBest solution in Generation 2:", best_solution, "with Fitness:", fitness(best_solution))


Generation 2 (P₂):
Offspring 1: (3, 2, 3) -> Binary: 111011 -> Fitness: 48
Offspring 2: (1, 2, 1) -> Binary: 011001 -> Fitness: 12
Offspring 3: (3, 0, 0) -> Binary: 110000 -> Fitness: 4
Offspring 4: (1, 0, 1) -> Binary: 010001 -> Fitness: 4

Best solution in Generation 2: (3, 2, 3) with Fitness: 48


## Summary and Conclusion

In this lesson, we:
- Introduced the basics of genetic algorithms.
- Defined our problem with variables \(x\), \(y\), and \(z\) (each in the range \([0,3]\)) and a new fitness function:

  \[
  F(x, y, z) = (x+1) \times (y+1) \times (z+1)
  \]
  
- Represented each individual as a 6-bit chromosome (2 bits per variable).
- Implemented the GA operators: **selection** (fixed pairing for demonstration), **two-point crossover**, and **mutation**.
- Evolved the population over two generations and identified the best solution in Generation 2.




## Modification of this basic implementation of the genetic algorithm

The first modification we will ionclude has to do with the selection of the parents. We will use the roulette wheel selection.

In [48]:
def roulette_wheel_selection(population, fitnesses):
    """
    Perform one roulette-wheel selection to pick a single individual (in binary).
    population: list of binary strings
    fitnesses: list of corresponding fitness values
    """
    total_fit = sum(fitnesses)
    pick = random.uniform(0, total_fit)
    current = 0
    
    for i, f in enumerate(fitnesses):
        current += f
        if current >= pick:
            return population[i]
    return population[-1] 

def select_two_distinct_parents(population, fitnesses):
    """
    Use roulette wheel selection twice, ensuring the second parent is different.
    """
    parent_bin_1 = roulette_wheel_selection(population, fitnesses)
    parent_bin_2 = roulette_wheel_selection(population, fitnesses)
    
    # Re-draw if we picked the same chromosome
    while parent_bin_2 == parent_bin_1:
        parent_bin_2 = roulette_wheel_selection(population, fitnesses)
    
    return parent_bin_1, parent_bin_2

We will increase the initial population size and change the fitness function

In [49]:
def fitness(individual):
    """
    F(x, y, z) = (x+1)^2 - 2*(y+1)^2 + 3*(z+1)^2
    x, y, z in [0..3].
    """
    x, y, z = individual
    return (x+1)**2 - 2*(y+1)**2 + 3*(z+1)**2

In [50]:
initial_population_decimal = [
    (1, 0, 3),  # from original
    (1, 2, 0),  # from original
    (1, 3, 1),  # from original
    (1, 0, 0),  # from original
    (0, 0, 0),  # new
    (3, 3, 3),  # new
    (2, 1, 2),  # new
    (0, 3, 1)   # new
]

In [51]:
# Convert them all to binary
population = [encode_individual(ind) for ind in initial_population_decimal]

print("=== Initial Population (Generation 0) ===")
for i, bin_str in enumerate(population):
    dec = decode_individual(bin_str)
    print(f"Ind {i+1}: {dec}, bin={bin_str}, fitness={fitness(dec)}")
print("")

=== Initial Population (Generation 0) ===
Ind 1: (1, 0, 3), bin=010011, fitness=50
Ind 2: (1, 2, 0), bin=011000, fitness=-11
Ind 3: (1, 3, 1), bin=011101, fitness=-16
Ind 4: (1, 0, 0), bin=010000, fitness=5
Ind 5: (0, 0, 0), bin=000000, fitness=2
Ind 6: (3, 3, 3), bin=111111, fitness=32
Ind 7: (2, 1, 2), bin=100110, fitness=28
Ind 8: (0, 3, 1), bin=001101, fitness=-19



Now we create a GA loop to do all the generations

In [54]:
# Set a random seed for reproducibility (optional)
random.seed(42)

# Number of bits needed per variable (0-3 => 2 bits each)
BITS_PER_VAR = 2

# Total length of chromosome: x, y, z => 3 variables => 6 bits
CHROMO_LENGTH = 3 * BITS_PER_VAR

# Population size
POP_SIZE = 8

# Number of generations to run
NUM_GENERATIONS = 5

# Probability of crossover
CROSSOVER_RATE = 0.8

for gen in range(1, NUM_GENERATIONS + 1):
    print(f"=== Generation {gen} ===")
    
    # Compute fitness of current population (decode each and evaluate)
    decoded_pop = [decode_individual(p) for p in population]
    pop_fitnesses = [fitness(ind) for ind in decoded_pop]
    
    new_population = []
    
    # We'll create the same population size with pairs of offspring
    # i.e., for 8 individuals => we need 4 pairs => 4 crossovers => 8 offspring
    for pair_index in range(4):  # 8 / 2 = 4 pairs
        # SELECT PARENTS via roulette wheel
        parent_bin_1, parent_bin_2 = select_two_distinct_parents(population, pop_fitnesses)
        
        # Print which parents we got (decode to show user)
        p1_dec = decode_individual(parent_bin_1)
        p2_dec = decode_individual(parent_bin_2)
        print(f"  Pair {pair_index+1} - Parent1: {p1_dec}, fitness={fitness(p1_dec)} | "
              f"Parent2: {p2_dec}, fitness={fitness(p2_dec)}")
        
        # CROSSOVER
        offspring_bin_1, offspring_bin_2 = two_point_crossover(parent_bin_1, parent_bin_2)
        
        # MUTATE
        offspring_bin_1 = mutate(offspring_bin_1)
        offspring_bin_2 = mutate(offspring_bin_2)
        
        # Add them to the new population
        new_population.append(offspring_bin_1)
        new_population.append(offspring_bin_2)
    
    # Update population
    population = new_population
    
    # Print out new population's individuals & fitness
    print("  -- New Population --")
    decoded_pop = [decode_individual(p) for p in population]
    pop_fitnesses = [fitness(ind) for ind in decoded_pop]
    
    for i, (dp, f) in enumerate(zip(decoded_pop, pop_fitnesses)):
        print(f"    Ind {i+1}: {dp}, fitness={f}")
    
    # Identify best
    best_index = max(range(len(decoded_pop)), key=lambda i: pop_fitnesses[i])
    best_dec = decoded_pop[best_index]
    print(f"  Best in Gen {gen}: {best_dec} with fitness={pop_fitnesses[best_index]}\n")

=== Generation 1 ===
  Pair 1 - Parent1: (0, 0, 3), fitness=47 | Parent2: (0, 0, 2), fitness=26
  Pair 2 - Parent1: (0, 0, 3), fitness=47 | Parent2: (0, 0, 2), fitness=26
  Pair 3 - Parent1: (2, 1, 2), fitness=28 | Parent2: (0, 0, 3), fitness=47
  Pair 4 - Parent1: (0, 0, 3), fitness=47 | Parent2: (2, 1, 2), fitness=28
  -- New Population --
    Ind 1: (1, 0, 3), fitness=50
    Ind 2: (1, 0, 2), fitness=29
    Ind 3: (2, 0, 3), fitness=55
    Ind 4: (2, 0, 2), fitness=34
    Ind 5: (3, 0, 2), fitness=41
    Ind 6: (0, 1, 2), fitness=20
    Ind 7: (2, 1, 3), fitness=49
    Ind 8: (3, 0, 2), fitness=41
  Best in Gen 1: (2, 0, 3) with fitness=55

=== Generation 2 ===
  Pair 1 - Parent1: (0, 1, 2), fitness=20 | Parent2: (2, 0, 3), fitness=55
  Pair 2 - Parent1: (1, 0, 3), fitness=50 | Parent2: (2, 1, 3), fitness=49
  Pair 3 - Parent1: (2, 0, 2), fitness=34 | Parent2: (1, 0, 3), fitness=50
  Pair 4 - Parent1: (2, 0, 3), fitness=55 | Parent2: (3, 0, 2), fitness=41
  -- New Population --
    

In [55]:
for gen in range(1, NUM_GENERATIONS + 1):
    print(f"=== Generation {gen} ===")
    
    # Evaluate current population
    decoded_pop = [decode_individual(p) for p in population]
    pop_fitnesses = [fitness(ind) for ind in decoded_pop]
    
    # Identify the best individual from the old population (elitism)
    best_old_index = max(range(len(decoded_pop)), key=lambda i: pop_fitnesses[i])
    best_old_ind_bin = population[best_old_index]
    best_old_ind_dec = decoded_pop[best_old_index]
    best_old_fit = pop_fitnesses[best_old_index]
    
    new_population = []
    
    # Create 8 offspring total (4 pairs)
    for pair_index in range(4):
        # Select two distinct parents
        parent_bin_1, parent_bin_2 = select_two_distinct_parents(population, pop_fitnesses)
        
        # Print parents info
        p1_dec = decode_individual(parent_bin_1)
        p2_dec = decode_individual(parent_bin_2)
        print(f"  Pair {pair_index+1} - Parent1: {p1_dec}, fitness={fitness(p1_dec)} | "
              f"Parent2: {p2_dec}, fitness={fitness(p2_dec)}")
        
        # Crossover
        offspring_bin_1, offspring_bin_2 = two_point_crossover(parent_bin_1, parent_bin_2)
        
        # Mutate
        offspring_bin_1 = mutate(offspring_bin_1)
        offspring_bin_2 = mutate(offspring_bin_2)
        
        # Add them to new population
        new_population.append(offspring_bin_1)
        new_population.append(offspring_bin_2)
    
    # Now apply elitism:
    # Decode new offspring to check their fitness
    decoded_new = [decode_individual(chromo) for chromo in new_population]
    new_fitnesses = [fitness(ind) for ind in decoded_new]
    
    # Find the best new fitness (to compare with the old best)
    best_new_fit = max(new_fitnesses)
    
    # If the old best is strictly better than the best in the new population,
    # replace the worst new individual with the old best.
    if best_old_fit > best_new_fit:
        # Find the index of the worst new individual
        worst_new_index = min(range(len(decoded_new)), key=lambda i: new_fitnesses[i])
        # Replace it with the old best individual's chromosome
        new_population[worst_new_index] = best_old_ind_bin
        decoded_new[worst_new_index] = best_old_ind_dec
        new_fitnesses[worst_new_index] = best_old_fit
        print(f"  [Elitism] Old best {best_old_ind_dec} (fitness={best_old_fit}) "
              "replaces worst new individual.")
    
    # Update population
    population = new_population
    
    # Print out new population details
    print("  -- New Population --")
    for i, (dp, f) in enumerate(zip(decoded_new, new_fitnesses)):
        print(f"    Ind {i+1}: {dp}, fitness={f}")
    
    # Identify the best solution in the new generation
    best_index = max(range(len(decoded_new)), key=lambda i: new_fitnesses[i])
    best_dec = decoded_new[best_index]
    print(f"  Best in Gen {gen}: {best_dec} with fitness={new_fitnesses[best_index]}\n")

=== Generation 1 ===
  Pair 1 - Parent1: (2, 1, 3), fitness=49 | Parent2: (2, 0, 3), fitness=55
  Pair 2 - Parent1: (2, 1, 3), fitness=49 | Parent2: (2, 0, 3), fitness=55
  Pair 3 - Parent1: (2, 1, 3), fitness=49 | Parent2: (2, 2, 2), fitness=18
  Pair 4 - Parent1: (2, 1, 3), fitness=49 | Parent2: (3, 1, 3), fitness=56
  [Elitism] Old best (3, 1, 3) (fitness=56) replaces worst new individual.
  -- New Population --
    Ind 1: (0, 0, 3), fitness=47
    Ind 2: (2, 1, 2), fitness=28
    Ind 3: (0, 0, 3), fitness=47
    Ind 4: (2, 3, 3), fitness=25
    Ind 5: (2, 3, 3), fitness=25
    Ind 6: (3, 1, 3), fitness=56
    Ind 7: (2, 1, 2), fitness=28
    Ind 8: (3, 1, 1), fitness=20
  Best in Gen 1: (3, 1, 3) with fitness=56

=== Generation 2 ===
  Pair 1 - Parent1: (2, 3, 3), fitness=25 | Parent2: (0, 0, 3), fitness=47
  Pair 2 - Parent1: (2, 1, 2), fitness=28 | Parent2: (2, 3, 3), fitness=25
  Pair 3 - Parent1: (2, 1, 2), fitness=28 | Parent2: (0, 0, 3), fitness=47
  Pair 4 - Parent1: (0, 0, 

In [42]:
import random

# For reproducible results (optional)
random.seed(42)

# Dictionary for integer-to-binary mapping (0..3 => 2 bits)
bin_map = {0: '00', 1: '01', 2: '10', 3: '11'}

def encode_individual(individual):
    return "".join(bin_map[val] for val in individual)

def decode_individual(binary_str):
    x = int(binary_str[0:2], 2)
    y = int(binary_str[2:4], 2)
    z = int(binary_str[4:6], 2)
    return (x, y, z)

def fitness(individual):
    """
    Example fitness:
    F(x, y, z) = (x+1)*(y+1)*(z+1)
    """
    x, y, z = individual
    return (x + 1) * (y + 1) * (z + 1)

def two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8):
    """
    Two-point crossover with extra printing to show the process.
    """
    # Decide whether crossover occurs
    rand_val = random.random()
    print(f"  [Crossover Check] rand={rand_val:.2f}, threshold={crossover_rate}")
    if rand_val > crossover_rate:
        print("  [No crossover] Offspring are copies of parents.")
        return parent1, parent2
    
    # Slicing segments
    seg1_p1 = parent1[:cp1]
    seg2_p1 = parent1[cp1:cp2]
    seg3_p1 = parent1[cp2:]

    seg1_p2 = parent2[:cp1]
    seg2_p2 = parent2[cp1:cp2]
    seg3_p2 = parent2[cp2:]

    print(f"  [Crossover] Parent1={parent1}, Parent2={parent2}")
    print(f"    Segments P1: [{seg1_p1}]|[{seg2_p1}]|[{seg3_p1}]")
    print(f"    Segments P2: [{seg1_p2}]|[{seg2_p2}]|[{seg3_p2}]")

    # Recombine
    offspring1 = seg1_p1 + seg2_p2 + seg3_p1
    offspring2 = seg1_p2 + seg2_p1 + seg3_p2

    print(f"    => Offspring1 before mutation: {offspring1}")
    print(f"    => Offspring2 before mutation: {offspring2}")

    return offspring1, offspring2

def mutate(binary_str):
    """
    Flip exactly one random bit in the 6-bit string, with printed details.
    """
    bits = list(binary_str)
    mutation_index = random.randint(0, len(bits) - 1)
    original_bit = bits[mutation_index]
    bits[mutation_index] = '1' if bits[mutation_index] == '0' else '0'
    mutated_str = "".join(bits)
    print(f"  [Mutation] Index={mutation_index}, from {original_bit} to {bits[mutation_index]},"
          f" result={mutated_str}")
    return mutated_str

# -------------------------
# DEFINE OUR POPULATION
# -------------------------
population_decimal = [
    (1, 0, 3),
    (1, 2, 0),
    (1, 3, 1),
    (1, 0, 0),
]

population_binary = [encode_individual(ind) for ind in population_decimal]

print("=== Initial Population (Decimal -> Binary -> Fitness) ===")
for i, bin_str in enumerate(population_binary):
    dec = decode_individual(bin_str)
    print(f"Individual {i+1}: Decimal={dec}, Binary={bin_str}, Fitness={fitness(dec)}")
print()

# -------------------------
# GENERATION 1
# -------------------------
print("=== Generation 1 (P₁) ===")

# Pair 1: Ind1 & Ind3 (indices 0, 2)
parent1 = population_binary[0]  # e.g., '010011'
parent2 = population_binary[2]  # e.g., '011101'

print(f"\n-- Pair 1 --")
offspring1, offspring2 = two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8)

# Apply mutation on both offspring
offspring1 = mutate(offspring1)
offspring2 = mutate(offspring2)

# Pair 2: Ind2 & Ind4 (indices 1, 3)
parent3 = population_binary[1]  # e.g., '011000'
parent4 = population_binary[3]  # e.g., '010000'

print(f"\n-- Pair 2 --")
offspring3, offspring4 = two_point_crossover(parent3, parent4, cp1=2, cp2=4, crossover_rate=0.8)
offspring3 = mutate(offspring3)
offspring4 = mutate(offspring4)

# New Gen 1
P1_binary = [offspring1, offspring2, offspring3, offspring4]
P1_decimal = [decode_individual(ind) for ind in P1_binary]

print("\n==> New Generation 1 (P₁) Offspring")
for i, ind in enumerate(P1_decimal, start=1):
    print(f"Offspring {i}: Dec={ind} -> Bin={P1_binary[i-1]} -> Fitness={fitness(ind)}")

# -------------------------
# GENERATION 2
# -------------------------
print("\n=== Generation 2 (P₂) ===")

# For new pairings, let's say:
# Pair 1: Offspring 1 & Offspring 4  (indices 0, 3)
# Pair 2: Offspring 2 & Offspring 3  (indices 1, 2)

parent1 = P1_binary[0]
parent2 = P1_binary[3]

print(f"\n-- Pair 1 --")
offspring1, offspring2 = two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8)
offspring1 = mutate(offspring1)
offspring2 = mutate(offspring2)

parent3 = P1_binary[1]
parent4 = P1_binary[2]

print(f"\n-- Pair 2 --")
offspring3, offspring4 = two_point_crossover(parent3, parent4, cp1=2, cp2=4, crossover_rate=0.8)
offspring3 = mutate(offspring3)
offspring4 = mutate(offspring4)

# New Gen 2
P2_binary = [offspring1, offspring2, offspring3, offspring4]
P2_decimal = [decode_individual(ind) for ind in P2_binary]

print("\n==> New Generation 2 (P₂) Offspring")
for i, ind in enumerate(P2_decimal, start=1):
    print(f"Offspring {i}: Dec={ind} -> Bin={P2_binary[i-1]} -> Fitness={fitness(ind)}")

# Identify and display best solution in Generation 2
best_solution = max(P2_decimal, key=fitness)
print(f"\nBest solution in Generation 2: {best_solution} with Fitness={fitness(best_solution)}")


=== Initial Population (Decimal -> Binary -> Fitness) ===
Individual 1: Decimal=(1, 0, 3), Binary=010011, Fitness=8
Individual 2: Decimal=(1, 2, 0), Binary=011000, Fitness=6
Individual 3: Decimal=(1, 3, 1), Binary=011101, Fitness=16
Individual 4: Decimal=(1, 0, 0), Binary=010000, Fitness=2

=== Generation 1 (P₁) ===

-- Pair 1 --
  [Crossover Check] rand=0.64, threshold=0.8
  [Crossover] Parent1=010011, Parent2=011101
    Segments P1: [01]|[00]|[11]
    Segments P2: [01]|[11]|[01]
    => Offspring1 before mutation: 011111
    => Offspring2 before mutation: 010001
  [Mutation] Index=0, from 0 to 1, result=111111
  [Mutation] Index=5, from 1 to 0, result=010000

-- Pair 2 --
  [Crossover Check] rand=0.28, threshold=0.8
  [Crossover] Parent1=011000, Parent2=010000
    Segments P1: [01]|[10]|[00]
    Segments P2: [01]|[00]|[00]
    => Offspring1 before mutation: 010000
    => Offspring2 before mutation: 011000
  [Mutation] Index=1, from 1 to 0, result=000000
  [Mutation] Index=1, from 1 to

## Solution to tutorial exercise: 

In [43]:
import random

# Optional: fix the seed for reproducibility
random.seed(42)

# Encode/Decode mapping (0..3 -> 2 bits)
bin_map = {0: '00', 1: '01', 2: '10', 3: '11'}

def encode_individual(individual):
    return "".join(bin_map[val] for val in individual)

def decode_individual(binary_str):
    x = int(binary_str[0:2], 2)
    y = int(binary_str[2:4], 2)
    z = int(binary_str[4:6], 2)
    return (x, y, z)

def fitness(individual):
    """
    New fitness function:
    F(x, y, z) = 5*x^2 + y^2 - (x*y*z) + 3
    for x, y, z in [0..3].
    """
    x, y, z = individual
    return 5*(x**2) + (y**2) - (x*y*z) + 3

def two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8):
    """
    Perform two-point crossover on two 6-bit binary strings.
    """
    if random.random() > crossover_rate:
        # No crossover => offspring are copies of the parents
        return parent1, parent2
    
    seg1_p1 = parent1[:cp1]
    seg2_p1 = parent1[cp1:cp2]
    seg3_p1 = parent1[cp2:]

    seg1_p2 = parent2[:cp1]
    seg2_p2 = parent2[cp1:cp2]
    seg3_p2 = parent2[cp2:]
    
    offspring1 = seg1_p1 + seg2_p2 + seg3_p1
    offspring2 = seg1_p2 + seg2_p1 + seg3_p2
    return offspring1, offspring2

def mutate(binary_str):
    """
    Flip exactly one random bit in the 6-bit string.
    """
    bits = list(binary_str)
    mutation_index = random.randint(0, len(bits) - 1)
    bits[mutation_index] = '1' if bits[mutation_index] == '0' else '0'
    return "".join(bits)

# -------------------------
# DEFINE OUR POPULATION
# -------------------------
population_decimal = [
    (1, 0, 3),
    (1, 2, 0),
    (1, 3, 1),
    (1, 0, 0),
]

population_binary = [encode_individual(ind) for ind in population_decimal]

print("=== Initial Population (Decimal -> Binary -> Fitness) ===")
for i, bin_str in enumerate(population_binary):
    dec = decode_individual(bin_str)
    print(f"Individual {i+1}: Decimal={dec}, Binary={bin_str}, Fitness={fitness(dec)}")
print()

# -------------------------
# GENERATION 1
# -------------------------
print("=== Generation 1 (P₁) ===")

# Pair 1: Ind1 & Ind3 (indices 0, 2)
parent1 = population_binary[0]
parent2 = population_binary[2]
offspring1, offspring2 = two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8)
offspring1 = mutate(offspring1)
offspring2 = mutate(offspring2)

# Pair 2: Ind2 & Ind4 (indices 1, 3)
parent3 = population_binary[1]
parent4 = population_binary[3]
offspring3, offspring4 = two_point_crossover(parent3, parent4, cp1=2, cp2=4, crossover_rate=0.8)
offspring3 = mutate(offspring3)
offspring4 = mutate(offspring4)

# New Gen 1
P1_binary = [offspring1, offspring2, offspring3, offspring4]
P1_decimal = [decode_individual(ind) for ind in P1_binary]

print("\n==> New Generation 1 (P₁) Offspring")
for i, ind in enumerate(P1_decimal, start=1):
    print(f"Offspring {i}: Dec={ind} -> Bin={P1_binary[i-1]} -> Fitness={fitness(ind)}")

# -------------------------
# GENERATION 2
# -------------------------
print("\n=== Generation 2 (P₂) ===")

# Pair 1: Offspring 1 & Offspring 4  (indices 0, 3)
parent1 = P1_binary[0]
parent2 = P1_binary[3]
offspring1, offspring2 = two_point_crossover(parent1, parent2, cp1=2, cp2=4, crossover_rate=0.8)
offspring1 = mutate(offspring1)
offspring2 = mutate(offspring2)

# Pair 2: Offspring 2 & Offspring 3  (indices 1, 2)
parent3 = P1_binary[1]
parent4 = P1_binary[2]
offspring3, offspring4 = two_point_crossover(parent3, parent4, cp1=2, cp2=4, crossover_rate=0.8)
offspring3 = mutate(offspring3)
offspring4 = mutate(offspring4)

# New Gen 2
P2_binary = [offspring1, offspring2, offspring3, offspring4]
P2_decimal = [decode_individual(ind) for ind in P2_binary]

print("\n==> New Generation 2 (P₂) Offspring")
for i, ind in enumerate(P2_decimal, start=1):
    print(f"Offspring {i}: Dec={ind} -> Bin={P2_binary[i-1]} -> Fitness={fitness(ind)}")

# Identify and display best solution in Generation 2
best_solution = max(P2_decimal, key=fitness)
print(f"\nBest solution in Generation 2: {best_solution}, Fitness={fitness(best_solution)}")


=== Initial Population (Decimal -> Binary -> Fitness) ===
Individual 1: Decimal=(1, 0, 3), Binary=010011, Fitness=8
Individual 2: Decimal=(1, 2, 0), Binary=011000, Fitness=12
Individual 3: Decimal=(1, 3, 1), Binary=011101, Fitness=14
Individual 4: Decimal=(1, 0, 0), Binary=010000, Fitness=8

=== Generation 1 (P₁) ===

==> New Generation 1 (P₁) Offspring
Offspring 1: Dec=(3, 3, 3) -> Bin=111111 -> Fitness=30
Offspring 2: Dec=(1, 0, 0) -> Bin=010000 -> Fitness=8
Offspring 3: Dec=(0, 0, 0) -> Bin=000000 -> Fitness=3
Offspring 4: Dec=(0, 2, 0) -> Bin=001000 -> Fitness=7

=== Generation 2 (P₂) ===

==> New Generation 2 (P₂) Offspring
Offspring 1: Dec=(3, 2, 2) -> Bin=111010 -> Fitness=40
Offspring 2: Dec=(0, 3, 1) -> Bin=001101 -> Fitness=12
Offspring 3: Dec=(3, 0, 0) -> Bin=110000 -> Fitness=48
Offspring 4: Dec=(0, 0, 2) -> Bin=000010 -> Fitness=3

Best solution in Generation 2: (3, 0, 0), Fitness=48


In [2]:
import random

# Optional: fix the seed for reproducibility
random.seed(42)

# -------------------------
# PARAMETERS & STUDENT DIGITS
# -------------------------
# Replace these with the corresponding digits from your student ID:
A = 5  # 4th last digit
B = 2  # 3rd last digit
C = 3  # 2nd last digit
D = 4  # last digit

# GA parameters
POPULATION_SIZE = 6
CROSSOVER_RATE = 1.0
MUTATION_RATE = 0.1  # probability that an offspring will undergo a mutation

# -------------------------
# BINARY ENCODING/DECODING (4 bits per variable)
# -------------------------
def encode_individual(individual):
    """
    Encodes a tuple (x, y, z) where each value is in [0,15] 
    as a 12-bit string (4 bits per variable).
    """
    return "".join(format(val, '04b') for val in individual)

def decode_individual(binary_str):
    """
    Decodes a 12-bit string into a tuple (x, y, z).
    """
    return (int(binary_str[0:4], 2), int(binary_str[4:8], 2), int(binary_str[8:12], 2))

# -------------------------
# OBJECTIVE FUNCTION (Fitness)
# -------------------------
def fitness(individual):
    """
    Objective function to maximise:
    
         F(x, y, z) = A*x^2 + B*y^2 - C*2*z + 5*D*x*y*z + 7
    
    where A, B, C, and D are constants (from your student ID digits).
    """
    x, y, z = individual
    return A * (x ** 2) + B * (y ** 2) - C * 2 * z + 5 * D * x * y * z + 7

# -------------------------
# GENETIC OPERATORS
# -------------------------
def two_point_crossover(parent1, parent2, cp1=4, cp2=8, crossover_rate=CROSSOVER_RATE):
    """
    Perform two-point crossover on two 12-bit binary strings.
    cp1 and cp2 are the cut points (here chosen so that the
    three genes, each of 4 bits, remain distinct).
    """
    if random.random() > crossover_rate:
        # No crossover: offspring are copies of the parents.
        return parent1, parent2

    offspring1 = parent1[:cp1] + parent2[cp1:cp2] + parent1[cp2:]
    offspring2 = parent2[:cp1] + parent1[cp1:cp2] + parent2[cp2:]
    return offspring1, offspring2

def mutate(binary_str, mutation_rate=MUTATION_RATE):
    """
    With probability mutation_rate, flip one random bit in the 12-bit string.
    """
    if random.random() < mutation_rate:
        bits = list(binary_str)
        mutation_index = random.randint(0, len(bits) - 1)
        bits[mutation_index] = '1' if bits[mutation_index] == '0' else '0'
        return "".join(bits)
    return binary_str

# -------------------------
# SELECTION OPERATOR
# -------------------------
def select_population(population):
    """
    Selects 6 individuals for reproduction from the current population by:
      - Selecting the best individual twice,
      - The worst individual once, and
      - Three other individuals randomly from the remaining ones.
    The input 'population' is a list of 12-bit binary strings.
    """
    evaluated = [(ind, fitness(decode_individual(ind))) for ind in population]
    # Sort by fitness in descending order (since we are maximising)
    evaluated_sorted = sorted(evaluated, key=lambda x: x[1], reverse=True)
    best_ind = evaluated_sorted[0][0]
    worst_ind = evaluated_sorted[-1][0]
    # Exclude the best and worst for random selection
    remaining = [ind for ind, fit in evaluated if ind not in (best_ind, worst_ind)]
    if len(remaining) >= 3:
        random_selected = random.sample(remaining, 3)
    else:
        random_selected = remaining
    selected_pool = [best_ind, best_ind, worst_ind] + random_selected
    return selected_pool

# -------------------------
# UTILITY: PRINT POPULATION DETAILS
# -------------------------
def print_population(population, generation_label):
    """
    Prints each individual's decimal representation, binary encoding,
    absolute fitness, and relative fitness (as a fraction of the total fitness).
    """
    decoded = [decode_individual(ind) for ind in population]
    fits = [fitness(ind) for ind in decoded]
    total_fit = sum(fits)
    print(f"=== {generation_label} ===")
    for i, (bin_str, dec, fit_val) in enumerate(zip(population, decoded, fits), start=1):
        rel_fit = fit_val / total_fit if total_fit != 0 else 0
        print(f"Individual {i}: Dec={dec}, Bin={bin_str}, Fitness={fit_val}, Relative Fitness={rel_fit:.3f}")
    print()

# -------------------------
# INITIAL POPULATION (P0)
# -------------------------
initial_population_decimal = [
    (0, 2, 4),
    (2, 1, 5),
    (10, 8, 12),
    (3, 5, 14),
    (2, 15, 9),
    (6, 5, 2),
]

P0_binary = [encode_individual(ind) for ind in initial_population_decimal]

print_population(P0_binary, "Initial Population (P0)")

# -------------------------
# GA PROCESS: NEXT 3 GENERATIONS
# -------------------------
current_population = P0_binary
num_generations = 3

for gen in range(1, num_generations + 1):
    print(f"--- Generation {gen} ---")
    
    # Selection: form the reproduction pool
    selected_pool = select_population(current_population)
    print("Selected individuals for reproduction:")
    print_population(selected_pool, f"Selection Pool for Generation {gen}")
    
    # Shuffle the selected pool to pair individuals randomly
    random.shuffle(selected_pool)
    
    # Pair the individuals randomly from the pool
    offspring_population = []
    for i in range(0, len(selected_pool), 2):
        parent1 = selected_pool[i]
        parent2 = selected_pool[i + 1]
        # Two-point crossover (with probability 1.0, so it always occurs)
        child1, child2 = two_point_crossover(parent1, parent2)
        # Apply mutation to each offspring
        child1 = mutate(child1)
        child2 = mutate(child2)
        offspring_population.extend([child1, child2])
    
    # New generation becomes the offspring
    current_population = offspring_population
    print_population(current_population, f"Generation {gen} Offspring (P{gen})")
    
    # Identify and display best solution in the generation
    decoded_current = [decode_individual(ind) for ind in current_population]
    best_individual = max(decoded_current, key=fitness)
    print(f"Best solution in Generation {gen}: {best_individual}, Fitness={fitness(best_individual)}\n")



=== Initial Population (P0) ===
Individual 1: Dec=(0, 2, 4), Bin=000000100100, Fitness=-9, Relative Fitness=-0.000
Individual 2: Dec=(2, 1, 5), Bin=001000010101, Fitness=199, Relative Fitness=0.006
Individual 3: Dec=(10, 8, 12), Bin=101010001100, Fitness=19763, Relative Fitness=0.629
Individual 4: Dec=(3, 5, 14), Bin=001101011110, Fitness=4218, Relative Fitness=0.134
Individual 5: Dec=(2, 15, 9), Bin=001011111001, Fitness=5823, Relative Fitness=0.185
Individual 6: Dec=(6, 5, 2), Bin=011001010010, Fitness=1425, Relative Fitness=0.045

--- Generation 1 ---
Selected individuals for reproduction:
=== Selection Pool for Generation 1 ===
Individual 1: Dec=(10, 8, 12), Bin=101010001100, Fitness=19763, Relative Fitness=0.436
Individual 2: Dec=(10, 8, 12), Bin=101010001100, Fitness=19763, Relative Fitness=0.436
Individual 3: Dec=(0, 2, 4), Bin=000000100100, Fitness=-9, Relative Fitness=-0.000
Individual 4: Dec=(2, 1, 5), Bin=001000010101, Fitness=199, Relative Fitness=0.004
Individual 5: Dec=(6